In [2]:
import matplotlib.pyplot as plt
import random
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from utils import *

# Explore derStandard.at Postings

In [5]:
df = load_data()

In [6]:
df.head()

,ID_Posting,ID_Posting_Parent,ID_CommunityIdentity,PostingHeadline,PostingComment,PostingCreatedAt,ID_Article,ArticlePublishingDate,ArticleTitle,ArticleChannel,ArticleRessortName,UserCommunityName,UserGender,UserCreatedAt
0,1041073586,1.041073e+09,671476,Das hat gestern bereits der Voggenhuber angefü...,schieder hatte dem inhaltlich nichts entgegenz...,2019-05-01 18:21:15.127,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Ravenspower,NaN,2018-04-14 13:42:28.470
1,1041073839,1.041073e+09,566938,NaN,...und meinen Bezirk bekommst du als Erbe mit.,2019-05-01 18:28:22.040,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,AlphaRomeo,m,2015-08-28 17:07:41.110
2,1041073872,1.041069e+09,669286,NaN,"Nein, bei der ÖVP/FPÖ genauso passiert. Ich wo...",2019-05-01 18:29:05.533,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Hpolditsch,NaN,2018-03-06 20:03:42.737
3,1041080734,1.041080e+09,671476,Sie haben doch nichts gefordert??,sie haben nur die regierung kritisiert. das di...,2019-05-01 22:37:56.010,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Ravenspower,NaN,2018-04-14 13:42:28.470
4,1041080828,NaN,671476,Heute wäre der perfekte Tag für die SPÖ gewese...,"ihr noch nicht erfülltes versprechen, den silb...",2019-05-01 22:42:06.310,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Ravenspower,NaN,2018-04-14 13:42:28.470


## First 1k users

In [46]:
selected_users = df['ID_CommunityIdentity'].unique()[:1000]

df_1k = df[df['ID_CommunityIdentity'].isin(selected_users)]
df_1k.shape

(155266, 14)

In [47]:
articles_per_user = df_1k.groupby('ID_CommunityIdentity')['ID_Article'].unique()
articles_per_user.head()

ID_CommunityIdentity
932     [2000102335365, 2000102333198, 2000102349261, ...
2183    [2000102263929, 2000102486297, 2000103145053, ...
3457    [2000102344996, 2000102620920, 2000103059819, ...
3481    [2000102351241, 2000102378773, 2000102370269, ...
4145    [2000102263929, 2000102257779, 2000102362845, ...
Name: ID_Article, dtype: object

### IOU

In [48]:
edge_weights = iou(selected_users, articles_per_user)

 10%|█         | 102/1000 [00:03<00:30, 29.65it/s]


KeyboardInterrupt: 

In [ ]:
edge_weights[:5, :5]

In [ ]:
edge_weights = edge_weights + edge_weights.T

In [ ]:
np.median(edge_weights.flatten())

In [ ]:
plt.hist(edge_weights.flatten(), bins=100, log=True)

In [ ]:
graph =  build_graph(selected_users, edge_weights, 0)

In [ ]:
# save graph
nx.write_gexf(graph, '../graphs/iou_graph.gexf')

In [ ]:
graph = nx.read_gexf('../graphs/iou_graph.gexf')

### IOM

In [ ]:
edge_weights = iom(selected_users, articles_per_user)

In [ ]:
# Create an empty graph
graph = build_graph(selected_users, edge_weights, 0)

In [ ]:
# save graph
nx.write_gexf(graph, '../graphs/iom_graph.gexf')

Only take the top 10\% of the edges

In [ ]:
# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

In [ ]:
nx.write_gexf(graph, '../graphs/iom_graph_10.gexf')

## Random Users >5 comments IOM

In [ ]:
df = load_data()

In [ ]:
# Get unique articles per user
articles_per_user = df.groupby('ID_CommunityIdentity')['ID_Article'].agg(set)
articles_per_user = articles_per_user[articles_per_user.apply(len) > 5].apply(list)

In [ ]:
# select users with more than 10 comments
selected_users = articles_per_user.index.to_list()
selected_users = random.sample(selected_users, k=1000)

In [ ]:
edge_weights = iom(selected_users, articles_per_user)

In [ ]:
plt.hist(edge_weights.flatten(), bins=100, log=True)

In [ ]:
# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

# Create an empty graph
graph = build_graph(selected_users, edge_weights, threshold)


In [ ]:
nx.write_gexf(graph, '../graphs/iom_graph.gexf')

## Filter By Channel

In [49]:
df = load_data()

In [50]:
df['ArticleChannel'].value_counts()

ArticleChannel
Inland           188069
Panorama          98391
Meinung           96008
Web               59589
International     55911
Wirtschaft        50755
Sport             40396
Etat              34890
Kultur            24013
User              12760
Wissenschaft      12565
Lifestyle         11124
Gesundheit        10490
Bildung            7942
dieStandard        7527
AutoMobil          6366
Immobilien         5071
Diverses           4597
Zukunft            4228
Reisen             3285
Familie            2658
Karriere           2459
Name: count, dtype: int64

In [51]:
df, articles_per_user, selected_users = filter_users(df, ['Sport', 'Wissenschaft'], min_articles= 5)

In [52]:
edge_weights = iom(selected_users, articles_per_user)

100%|██████████| 1239/1239 [00:07<00:00, 159.35it/s]


In [53]:
# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

# Create an empty graph
graph = build_graph(selected_users, edge_weights, threshold)


In [54]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Sport           1014
Wissenschaft     225
Name: count, dtype: int64


In [55]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')

In [56]:
nx.write_gexf(graph, '../graphs/iom_graph_sport_wissenschaft.gexf')

### Using Wissenschaft, Sport and AutoMobil

In [57]:
df = load_data()

df, articles_per_user, selected_users = filter_users(df, ['Wissenschaft', 'Sport', 'AutoMobil'], min_articles= 5)

In [58]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 1362/1362 [00:10<00:00, 134.44it/s]


In [59]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Sport           1035
Wissenschaft     249
AutoMobil         78
Name: count, dtype: int64


In [60]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')

In [61]:
nx.write_gexf(graph, '../graphs/iom_graph_sport_wissenschaft_automobil.gexf')

### Using Sport, Kultur

In [63]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Kultur', 'Sport'], min_articles=5)

In [64]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 1465/1465 [00:14<00:00, 98.42it/s] 


In [65]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Sport     1004
Kultur     461
Name: count, dtype: int64


In [66]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_sport_kultur.gexf')

### Using Karriere, Familie

In [67]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Karriere', 'Familie'], min_articles=5)

In [68]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 25/25 [00:00<00:00, 4628.66it/s]


In [69]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Karriere    16
Familie      9
Name: count, dtype: int64


In [70]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_karriere_familie.gexf')

### Using Gesundheit, Immobilien

In [71]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Gesundheit', 'Immobilien'], min_articles=5)

In [72]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 161/161 [00:00<00:00, 1309.21it/s]


In [73]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Gesundheit    112
Immobilien     49
Name: count, dtype: int64


In [74]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_gesundheit_immobilien.gexf')

### Using Sport, Karriere

In [75]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Karriere', 'Sport'], min_articles=5)

In [76]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 973/973 [00:05<00:00, 163.63it/s]


In [77]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Sport       953
Karriere     20
Name: count, dtype: int64


In [78]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_karriere_sport.gexf')

### Using Kultur, Wirtschaft

In [83]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Kultur', 'Wirtschaft'], min_articles=5)

In [84]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 1860/1860 [00:20<00:00, 89.61it/s] 


In [85]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Wirtschaft    1364
Kultur         496
Name: count, dtype: int64


In [86]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_kultur_wirtschaft.gexf')

### Using Familie, International

In [87]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Familie', 'International'], min_articles=5)

In [88]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 1272/1272 [00:08<00:00, 143.90it/s]


In [89]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
International    1255
Familie            17
Name: count, dtype: int64


In [90]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_familie_international.gexf')

### Using Reisen, Wissenschaft

In [91]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Reisen', 'Wissenschaft'], min_articles=5)

In [92]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 262/262 [00:00<00:00, 1022.80it/s]


In [93]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Wissenschaft    239
Reisen           23
Name: count, dtype: int64


In [94]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_reisen_wissenschaft.gexf')

### Using Lifestyle, Gesundheit

In [95]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Lifestyle', 'Gesundheit'], min_articles=5)

In [96]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 322/322 [00:00<00:00, 1002.89it/s]


In [97]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Lifestyle     181
Gesundheit    141
Name: count, dtype: int64


In [98]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_lifestyle_gesundheit.gexf')

### Using Bildung, Wissenschaft

In [99]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Bildung', 'Wissenschaft'], min_articles=5)

In [100]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 315/315 [00:00<00:00, 823.25it/s]


In [101]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Wissenschaft    246
Bildung          69
Name: count, dtype: int64


In [102]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_bidlung_wissenschaft.gexf')

### Using Lifestyle, Gesundheit, Familie

In [103]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Lifestyle','Gesundheit', 'Familie'], min_articles=5)

In [104]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 109/109 [00:00<00:00, 3778.50it/s]


In [105]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Gesundheit    97
Familie       12
Name: count, dtype: int64


In [106]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_lifestyle_graph_gesundheit_familie.gexf')

### Using Etat, Inland

In [107]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Etat', 'Inland'], min_articles=5)

In [108]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 4798/4798 [02:21<00:00, 33.91it/s] 


In [110]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Inland    4490
Etat       308
Name: count, dtype: int64
most_common
Inland    4490
Etat       308
Name: count, dtype: int64


In [111]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_etat_inland.gexf')

### Using User, Meinung

In [112]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['User', 'Meinung'], min_articles=5)

In [113]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 2350/2350 [00:28<00:00, 83.00it/s] 


In [114]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Meinung    2274
User         76
Name: count, dtype: int64


In [115]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_meinung_user.gexf')

### Using Inland, Wirtschaft, International

In [130]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Inland','Wirtschaft', 'International'], min_articles=5)

In [131]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 5795/5795 [04:53<00:00, 19.72it/s] 


In [132]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Inland           4721
International     540
Wirtschaft        534
Name: count, dtype: int64


In [133]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_inland_wirtschaft_international.gexf')

### Using AutoMobil, Immobilien

In [126]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['AutoMobil', 'Immobilien'], min_articles=5)

In [127]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 81/81 [00:00<00:00, 2828.96it/s]


In [128]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
AutoMobil     49
Immobilien    32
Name: count, dtype: int64


In [129]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_automobil_immobilien.gexf')

### Using Bildung, Zukunft

In [122]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Bildung', 'Zukunft'], min_articles=5)

In [123]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 59/59 [00:00<00:00, 6458.16it/s]


In [124]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Bildung    43
Zukunft    16
Name: count, dtype: int64


In [125]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_bildung_zukunft.gexf')

### Using all except inland, panorama, meinung

In [7]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Web', 'International','Wirtschaft','Sport','Etat','Kultur','User','Wissenschaft','Lifestyle','Gesundheit','Bildung','dieStandard','AutoMobil','Immobilien','Diverses','Zukunft','Reisen','Familie','Karriere'], min_articles=5)

In [8]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 7146/7146 [02:21<00:00, 50.47it/s] 


In [9]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Web              1539
International    1286
Sport            1069
Wirtschaft       1059
Etat              794
Kultur            384
Gesundheit        157
Lifestyle         155
Bildung           110
Wissenschaft      109
AutoMobil         100
User               96
dieStandard        87
Immobilien         53
Diverses           49
Familie            32
Karriere           25
Reisen             22
Zukunft            20
Name: count, dtype: int64


In [10]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_alle_ex_InPaMe.gexf')

### Using all except inland

In [11]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Panorama','Meinung','Web', 'International','Wirtschaft','Sport','Etat','Kultur','User','Wissenschaft','Lifestyle','Gesundheit','Bildung','dieStandard','AutoMobil','Immobilien','Diverses','Zukunft','Reisen','Familie','Karriere'], min_articles=5)

In [12]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 9014/9014 [04:54<00:00, 30.56it/s] 


In [13]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Meinung          2191
Panorama         2063
Web              1117
Sport             928
International     774
Wirtschaft        584
Etat              456
Kultur            214
Gesundheit        121
Lifestyle         119
AutoMobil          77
User               65
Wissenschaft       65
Bildung            56
dieStandard        55
Immobilien         39
Diverses           28
Familie            21
Zukunft            15
Reisen             14
Karriere           12
Name: count, dtype: int64


In [14]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_alle_ex_In.gexf')

### Using all

In [15]:
df = load_data()
df, articles_per_user, selected_users = filter_users(df, ['Inland','Panorama','Meinung','Web', 'International','Wirtschaft','Sport','Etat','Kultur','User','Wissenschaft','Lifestyle','Gesundheit','Bildung','dieStandard','AutoMobil','Immobilien','Diverses','Zukunft','Reisen','Familie','Karriere'], min_articles=5)

In [16]:
edge_weights = iom(selected_users, articles_per_user)

# Calculate the threshold value for the top 10% of weights
threshold = np.percentile(edge_weights.flatten(), 95)

graph = build_graph(selected_users, edge_weights, threshold)

100%|██████████| 10319/10319 [08:47<00:00, 19.54it/s] 


In [17]:
df_category_count = most_common_category_per_user(df)
print(df_category_count['most_common'].value_counts())

most_common
Inland           4317
Panorama         1319
Meinung           998
Web               885
Sport             828
International     505
Wirtschaft        407
Etat              342
Kultur            133
Gesundheit        110
Lifestyle          85
AutoMobil          66
User               59
Wissenschaft       54
dieStandard        52
Bildung            40
Immobilien         35
Familie            22
Diverses           20
Zukunft            15
Reisen             15
Karriere           12
Name: count, dtype: int64


In [18]:
nx.set_node_attributes(graph, df_category_count['most_common'].to_dict(), 'category')
nx.write_gexf(graph, '../graphs/iom_graph_all.gexf')